In [ ]:
#load landsat images
!aws --no-sign-request s3 cp s3://pangeo-data-upload-oregon/icesat2/data-access-outputs/LC08_L1GT_217105_20171203_20171207_01_T2_B2.TIF .
!aws --no-sign-request s3 cp s3://pangeo-data-upload-oregon/icesat2/data-access-outputs/LC08_L1GT_217105_20171203_20171207_01_T2_B3.TIF .
!aws --no-sign-request s3 cp s3://pangeo-data-upload-oregon/icesat2/data-access-outputs/LC08_L1GT_217105_20171203_20171207_01_T2_B4.TIF .
# !aws --no-sign-request s3 cp s3://pangeo-data-upload-oregon/icesat2/data-access-outputs/LC08_L1GT_217105_20171203_20171207_01_T2_B8.TIF .

In [ ]:
# #need to add pytables to the environment in terminal b/c it has a prompt!
# !conda install -y pytables

In [ ]:
#import packages
import os
import numpy as np
# File manipulation
from glob import glob
import gdal
import h5py
import pyproj

import matplotlib.pyplot as plt
import matplotlib

import rasterio as rio
import pandas as pd
import geopandas as gpd
from rasterio.plot import show
from shapely.geometry import Point, Polygon

#from Fernando Paolo & Johan Nilsson's utilities
def transform_coord(proj1, proj2, x, y):
    """Transform coordinates from proj1 to proj2 (EPSG num)."""
    
    # Set full EPSG projection strings
    proj1 = pyproj.Proj("+init=EPSG:"+proj1)
    proj2 = pyproj.Proj("+init=EPSG:"+proj2)
    
    # Convert coordinates
    return pyproj.transform(proj1, proj2, x, y)

In [ ]:
#load and stack the Landsat bands
image1_bands = glob("*.TIF")
image1_bands.sort()

image1_blue = np.squeeze(rio.open(image1_bands[0]).read())
image1_red = np.squeeze(rio.open(image1_bands[2]).read())
image1_green = np.squeeze(rio.open(image1_bands[1]).read())
#image1_nir = np.squeeze(rio.open(image1_bands[3]).read())
    
#Normalize bands into 0.0 - 1.0 scale
def normalize(array):
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

# Normalize band DN
blue = normalize(image1_blue)
red = normalize(image1_red)
green = normalize(image1_green)
#nir = normalize(image1_nir)

#ndwi = (green - nir)/(green - nir)
# Stack bands
rgb = np.dstack((red, green, blue))
#print(nrg.shape())
# View the color composite
rgb = rgb.astype(float)
#plt.imshow(rgb)
#es.plot_bands(image1_blue[0],title="Landsat Cropped Band 4\nColdsprings Fire Scar",cmap="Greys_r")

In [ ]:
#load metadata
tile_path = 'LC08_L1GT_217105_20171203_20171207_01_T2_B2.TIF'
Raster = gdal.Open(tile_path)
width = Raster.RasterXSize
height = Raster.RasterYSize
gt = Raster.GetGeoTransform()
array = Raster.ReadAsArray()

# Pixel numbers
x = np.arange(0, width)
y = np.arange(0, height)

# Grid Cell Coordinates of upper left corner in EPSG:3031 UTM. 
imX = gt[0] + x * gt[1] 
imY = gt[3] + y * gt[5]

In [ ]:
# #plot the figure
# # %matplotlib widget
# %matplotlib inline
# fig = plt.figure(figsize=(12,10))
# ax = plt.imshow(rgb, extent = [np.min(imX), np.max(imX), np.min(imY), np.max(imY)])
# plt.show()

In [ ]:
#crop down the data more to just the RGI polygon
ATL_file=glob('/home/jovyan/crackup/Outputs_New/Edgeworth*ATL06*.h5')[0] #adding [0] at the end tells it to pull the first string from the list
print(ATL_file)
if 'ATL03' in ATL_file:
    f = h5py.File(ATL_file, 'r')
    beam = [k for k in f.keys() if k.startswith('gt')]
    beam = beam[1] #specify the beam
    data = {}
    data[beam] = {}
    data[beam]['heights'] = {}
    conf = data[beam]['heights']['signal_conf_ph'][:,3]
    lat = data[beam]['heights']['lat_ph']
    lon = data[beam]['heights']['lon_ph']
    x, y = transform_coord('4326','3031', lon, lat) #convert to Antarctic PS
    h = data[beam]['heights']['h_ph']
    f.close()
else:
    f = h5py.File(ATL_file, 'r')
    beam6 = [k for k in f.keys() if k.startswith('gt')]
    beam6 = beam6[1] #1,3,5 strong vs 0,2,4 weak for now
    data6 = {}
    data6[beam6] = {}
    data6[beam6]['land_ice_segments'] = {}
    lookfor = ['delta_time','h_li','h_li_sigma','latitude','longitude','segment_id','sigma_geo_h']
    for key,val in f[beam6]['land_ice_segments'].items():
       if key in lookfor:
            data6[beam6]['land_ice_segments'][key] = val[:]
#     print(data6[beam6]['land_ice_segments'].keys())
    lat = data6[beam6]['land_ice_segments']['latitude']
    lon = data6[beam6]['land_ice_segments']['longitude']
    x, y = transform_coord('4326','3031', lon, lat) #convert to Antarctic PS
    h = data6[beam6]['land_ice_segments']['h_li']
    f.close()


In [ ]:
#create a pandas dataframe
data_dict = {
    'latitude': lat,
    'longitude': lon,
    'PSy': y,
    'PSx': x,
    'elev': h}
df = pd.DataFrame.from_dict(data_dict)

#create a new column called geometry
df['geometry'] = list(zip(df['longitude'], df['latitude']))
df['geometry'] = df['geometry'].apply(Point)


In [ ]:
#create a geodataframe object
gdf = gpd.GeoDataFrame(df, crs={'init' :'epsg:4326'})

In [ ]:
#read in the area of interest polygon
if 'Edgeworth' in ATL_file:
    poly_gdf = gpd.read_file('/home/jovyan/crackup/shapefiles/edgeworth_polygons.shp')
    poly_gdf = poly_gdf.set_index('glac_id')
else:
    poly_gdf = gpd.read_file('/home/jovyan/crackup/shapefiles/georgeVI_meltpond-polygon.shp')
    poly_gdf = poly_gdf.set_index('Id_text')
poly_gdf.columns

In [ ]:
#merge all the polygons for the glacier (poly_gdf), find where the ICESat data (gdf) intersect the polygon, crop the ICESat data (ATL_roi_gdf)
one_poly = poly_gdf.unary_union
ATL_roi_gdf_idx = gdf.intersects(one_poly)
ATL_roi_gdf = gdf[ATL_roi_gdf_idx]

In [ ]:
# #ONLY DO THIS IF YOU HAVE MULTIPLE GLACIER POLYGONS: merge ICESat2 points w/ polygon
# poly_cols = ['area','glac_name','wgms_id','geometry']
# ATL_roi_gdf = gpd.sjoin(gdf, poly_gdf[poly_cols], op='intersects', how='inner')
# ATL_roi_gdf = ATL_roi_gdf.rename(columns={"index_right": "glac_name"})

In [ ]:
#pull info from joined data
print("Number of ATL points before:", gdf.shape[0])
print("Number of ATL points that intersect polygons:", ATL_roi_gdf.shape[0])

In [ ]:
import rasterio.plot
f, ax = plt.subplots()
ax.imshow(rgb, extent = [np.min(imX), np.max(imX), np.min(imY), np.max(imY)])
# gpd.plot(ax=ax, edgecolor='k', lw=0.5, alpha=0.1); #on the same axes, plot the glacier outlines
ATL_roi_gdf.plot(ax=ax, column='elev', cmap='RdBu', markersize=1, legend=True) #also plot the dhdt calculated before
# ax.set_xlim(bbox[[0,2]]);
# ax.set_ylim(bbox[[1,3]]);